<a href="https://colab.research.google.com/github/pandov/myitacademy-pandov/blob/master/FERCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %load_ext tensorboard
num_classes = 5
# ! sh colab.sh setup
# ! dvc pull -d stages/FER{num_classes}.dvc
# ! unzip "data/external/*.zip"
# ! dvc repro stages/FER{num_classes}.dvc

In [ ]:
# %%capture
import torch
from torchvision import transforms
# from catalyst.dl.utils import set_global_seed, prepare_cudnn
# from src.kcv import CrossValidation, Experiment, experiment_run
from src.utils import get_fer_names, get_cnn_fer_weights
# from src.callbacks import *
# from src.features import FER
from src.models import FERCNN
# set_global_seed(3)
# prepare_cudnn(deterministic=True)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# weight = get_cnn_fer_weights(num_classes).to(device)
# from torchsummary import summary
# model = FERCNN(5).to(device)
# summary(model, input_size=(1, 48, 48))

In [ ]:
class Ex1(Experiment):
    def model(self): return FERCNN(num_classes=num_classes)
    def criterion(self): return torch.nn.CrossEntropyLoss(weight=weight)
    def optimizer(self, model): return torch.optim.Adam(model.parameters(), lr=1e-3)
    def scheduler(self, optimizer): return torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)
class Ex2(Ex1):
    def optimizer(self, model): return torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-5)
class Ex3(Ex1):
    def optimizer(self, model): return torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-4)
class Ex4(Ex1):
    def optimizer(self, model): return torch.optim.Adam(model.parameters(), lr=4e-3, weight_decay=1e-4)
    def optimizer(self, model): return torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-5)
class Ex5(Ex3):
    def scheduler(self, optimizer): return torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.7)

In [ ]:
%tensorboard --logdir logs

In [ ]:
%%capture
experiments = [Ex1(), Ex2(), Ex3(), Ex4(), Ex5()]
class_names = get_fer_names(num_classes)
callbacks = [
    AccuracyCallback(num_classes=num_classes),
    AUCCallback(num_classes=num_classes, class_names=class_names),
    PrecisionRecallF1ScoreCallback(num_classes=num_classes, class_names=class_names),
    ConfusionMatrixCallback(num_classes=num_classes, class_names=class_names),
]
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((48, 48)),
    transforms.RandomPerspective(distortion_scale=0.2),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.1),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])
cross_val = CrossValidation(FER(num_classes, transform=transform), 5)
logdir = f'logs/FERCNN{num_classes}'
experiment_run(experiments, callbacks, cross_val, logdir, device, 'f1/_mean', 300, 250)

In [ ]:
! sh colab.sh save $logdir

In [ ]:
# ! rm -r logs/FERCNN{num_classes}